### For connecting and retrieving query results from MYSQL

In [ ]:
# pip install mysql-connector-python

In [ ]:
import mysql.connector
import pandas as pd

class mysqlconnector:
    
    def __init__(self, database = 'test'):
        try:
            connection = mysql.connector.connect(host = 'localhost',
                                                user = 'root',
                                                password = '',
                                                use_pure = True,
                                                database = database
                                                )
            if connection.is_connected:
                db_info = connection.get_server_info()
                print('connected to MYSQL server version', db_info)
                print('You are connected to the database:', database)
                self.connection = connection
        except Exception as e:
            print('Error while connecting to MYSQL', e)
            
    def execute(self, query, header = False):
        cursor = self.connection.cursor(buffered = True)
        cursor.execute(query)
        
        try:
            record = cursor.fetchall()
            if header:
                header = [i[0] for i in cursor.description]
                return {'header': header, 'record': record}
            else:
                return record
        except:
            pass
        
    def to_df(self, query):
        result = self.execute(query, header = True)
        df = pd.DataFrame(result['record'])
        df.columns = result['header']
        return df

In [ ]:
db = mysqlconnector('bank')

# EDA

In [ ]:
query = 'SELECT * FROM Loan JOIN Account USING(account_id);'
df = db.to_df(query)

In [ ]:
df.sample(10)

In [ ]:
# df_good for food loans
df_good = df[df['status'].isin(['A','C'])]
df_good.shape

In [ ]:
# df_bad for bad loans
df_bad = df[df['status'].isin(['B', 'D'])]
df_bad.shape

In [ ]:
import matplotlib.pyplot as plt

# To plot histogram of the loan amount
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))
df_good['amount'].hist(bins = 25, ax = ax1, label = 'good loans', color = 'grey')
df_bad['amount'].hist(bins =  25, ax = ax2, label = 'bad loans', color = 'red')

ax1.legend()
ax2.legend()

In [ ]:
# To plot histogram of the loan duration

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))
df_good['duration'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df_bad['duration'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend(loc = 9)
ax2.legend()

In [ ]:
# To plot histogram of the loan monthly installment

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))
df_good['payments'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df_bad['payments'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

In [ ]:
# To plot histogram of the loan location

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))
df_good['district_id'].hist(ax = ax1, label = 'good_loan', color = 'grey', bins = 25)
df_bad['district_id'].hist(ax = ax2, label = 'bad_loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

In [ ]:
# To rename columns date column in Loan and Account tables

# No need to run rename commands as the columns name have been changed in the database

query = """ALTER TABLE Loan 
        CHANGE COLUMN `date` Loan_date DATE;"""

df1 = db.to_df(query)

In [ ]:
query = 'SELECT * FROM Loan;'

df1 = db.to_df(query)
df1.head()

In [ ]:
query = 'ALTER TABLE Account CHANGE COLUMN `date` Account_date DATE;'
db.to_df(query)

In [ ]:
query = 'SELECT * FROM Loan JOIN Account USING(account_id)'
df = db.to_df(query)
df.head()

In [ ]:
df1 = df.copy()
# df1.rename(columns = {'date': 'loan_dates'})
df1.columns.duplicated()

In [ ]:
# feature creation from date of account opening and date of applying loan (DID : Dfference In Dates)

df['DID'] = df['Loan_date'] - df['Account_date']
df_good = df[df['status'].isin(['A', 'C'])]
df_bad = df[df['status'].isin(['B', 'D'])]

In [ ]:
# Converts timedelta64 to float64 as timedelta was not being able to use for plotting histogram
df_good['DID'].astype('timedelta64[D]')

In [ ]:
# Another way of converting the timedelta64 to the int64 for plotting histogram
df_good['DID'].dt.days

In [ ]:
# To plot the histogram of the time difference between opening the account and the taking the loan
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))
df_good['DID'].astype('timedelta64[D]').hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df_bad['DID'].astype('timedelta64[D]').hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

## Joining district along with the loan and account tables

In [ ]:
query = """SELECT * FROM Loan JOIN Account USING(account_id) JOIN District USING(district_id)"""

df1 = db.to_df(query)

In [ ]:
df1.sample(10)

In [ ]:
df1 = df1.rename(columns = {'A4': 'No_inhabitants', 'A11': 'Average_salary', 'A14': 'No_entrepreneur_per1000'})

df1['Average_unemployment_rate'] = df1[['A12', 'A13']].mean(axis = 1)
df1['Average_crime_rate'] = df1[['A15', 'A16']].mean(axis = 1)/df1['No_inhabitants']

df1.head()

In [ ]:
df1.columns

In [ ]:
df1_good = df1[df1['status'].isin(['A', 'C'])]
df1_bad = df1[df1['status'].isin(['B', 'D'])]

In [ ]:
# Histograme plot for Number of inhabitants

fig, (ax1,ax2) = plt.subplots(1, 2, figsize = (15,5))

df1_good['No_inhabitants'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df1_bad['No_inhabitants'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()


In [ ]:
# Histogram plot for average salary

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))
df1_good['Average_salary'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df1_bad['Average_salary'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

In [ ]:
# Histogram to plot for average unemployment rate 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))
df1_good['Average_unemployment_rate'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df1_bad['Average_unemployment_rate'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

In [ ]:
df1['Average_unemployment_rate']

In [ ]:
# Histogram for average crime rate

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))

df1_good['Average_crime_rate'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df1_bad['Average_crime_rate'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

## Matching order tabel with loan table

In [ ]:
query = 'SELECT * FROM Loan;'

df2 = db.to_df(query)

In [ ]:
df2.head()

In [ ]:
query = """
        SELECT account_id, amount Order_amount
        FROM `Order`
        WHERE account_id in (SELECT account_id FROM Loan);
        """

df3 = db.to_df(query)

In [ ]:
df3.head()

In [ ]:
# Datatype of Order_amount was 'Object' before changing it to 'float'

df3['Order_amount'] = df3['Order_amount'].astype('float')

In [ ]:
df3.groupby('account_id').mean()

In [ ]:
df4 = df2.set_index('account_id').join(df3.groupby('account_id').mean())
df4.head()

# or use pd.merge(df2, df3.groupby('account_id').mean(), on = 'account_id', how = 'outer').set_index('account_id')

In [ ]:
df4 = df4.rename(columns = {'Order_amount': 'Average order_amount'})
df4.head()

In [ ]:
df4_good = df4[df4['status'].isin(['A', 'C'])]
df4_bad = df4[df4['status'].isin(['B', 'D'])]

In [ ]:
# histogram plot for average order amount

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15,5))
df4_good['Average order_amount'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df4_bad['Average order_amount'].hist(ax= ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

## Combining loan table with transaction table

In [ ]:
query = """
        SELECT * 
        FROM Loan;"""

df5 = db.to_df(query)

In [ ]:
df5.head()

In [ ]:
query = """
        SELECT account_id, amount Trans_amount, balance Trans_balance
        FROM Trans WHERE account_id IN (SELECT account_id FROM Loan);
        """

df6 = db.to_df(query)

In [ ]:
df6.head()

In [ ]:
No_transaction = df6.groupby('account_id').count().iloc[:,1]
No_transaction.name = 'No_transaction'
No_transaction

In [ ]:
df6 = df6.groupby('account_id').mean()
df6.columns = ['Average_trans_amount', 'Average_trans_balance']
df6.head()

In [ ]:
df7 = df5.set_index('account_id').join(df6).join(No_transaction)
df7.sample(5)

In [ ]:
print(df5.shape, df6.shape, No_transaction.shape, df7.shape)

In [ ]:
df7_good = df7[df7['status'].isin(['A', 'C'])]
df7_bad = df7[df7['status'].isin(['B', 'D'])]

In [ ]:
# Histogram plot for average transaction amount

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))

df7_good['Average_trans_amount'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df7_bad['Average_trans_amount'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

In [ ]:
# Histogram plot for average balance after transaction

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))

df7_good['Average_trans_balance'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df7_bad['Average_trans_balance'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

In [ ]:
# Histogram for number of transaction

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))

df7_good['No_transaction'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df7_bad['No_transaction'].hist(ax = ax2, label = 'bad loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

## Combining Loan tabel with Credit card tabel

In [ ]:
query = """
        SELECT account_id, card_id, Card.type Card_type, status
        FROM Loan JOIN Desposition USING(account_id) LEFT JOIN Card USING(disp_id)
        """

# JOIN operation with USING gives only those rows in which the column mention in the USING matches.

df8 = db.to_df(query)
df8.sample(5)

In [ ]:
df8_card = df8[~df8['card_id'].isna()]

# FROM Loan JOIN Desposition USING(account_id) JOIN Card USING(disp_id); can also be used but it was not working

In [ ]:
# query = """
#         SELECT account_id, card_id, Card.type Card_type, status
#         FROM Loan JOIN Desposition USING(account_id) JOIN Card USING(disp_id);
#         """

# df9 = db.to_df(query)
# df9.sample(5)

In [ ]:
df8_good = df8[df8['status'].isin(['A', 'C'])]
df8_bad = df8[df8['status'].isin(['B', 'D'])]

In [ ]:
# Barchart for credit card type

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5)) 

df8_good[['Card_type', 'account_id']].groupby('Card_type', dropna = False).count().plot.bar(ax = ax1, color = 'grey', label = 'good loan', legend = False)
df8_bad[['Card_type', 'account_id']].groupby('Card_type', dropna = False).count().plot.bar(ax = ax2, color = 'red', label = 'bad loan', legend = False)

## Combing Account table and Client Table

In [ ]:
# Taking aprrox. 6 mins to run

query = """
        SELECT * 
        FROM Loan JOIN Desposition USING(account_id) 
        JOIN Client USING(Client_id) 
        JOIN District USING(district_id);
        """
df9 = db.to_df(query)

In [ ]:
df9.head()

In [ ]:
query = """
         SELECT account_id, district_id Acc_dist_id
         FROM Loan JOIN Account USING(account_id);
         """

df10 = db.to_df(query)

In [ ]:
df10.head()

In [ ]:
df11 = df9.set_index('account_id').join(df10.set_index('account_id'))

# or
# df12 = pd.merge(df9, df10, on = 'account_id', how = 'inner').set_index('account_id')

In [ ]:
df11.head()

In [ ]:
df11['type'].unique()

In [ ]:
df11 = df11[df11['type'] == 'OWNER']

In [ ]:
print(df11.shape, df11.index.unique().shape)

In [ ]:
df11['Same_district'] = df11['district_id'] == df11['Acc_dist_id']

In [ ]:
df11.head()

In [ ]:
df11['Loan_date'] = df11['Loan_date'].astype('datetime64')
df11['birth_date'] = df11['birth_date'].astype('datetime64')
df11['Owner_age'] = df11['Loan_date'] - df11['birth_date']

In [ ]:
df11['Owner_age']

In [ ]:
# To convert days value of timedelta64 type into year values of float64 type

import numpy as np

df11['Owner_age'] = df11['Owner_age']/np.timedelta64(1, 'Y')

In [ ]:
df11 = df11.reset_index()
df11_good = df11[df11['status'].isin(['A', 'C'])]
df11_bad = df11[df11['status'].isin(['B', 'D'])]

In [ ]:
df11_good.head()

In [ ]:
# Bar chart for client district and account district
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))

df11_good[['account_id', 'Same_district']].groupby('Same_district').count().plot.bar(ax = ax1, color = 'grey', legend = False)
df11_bad[['account_id', 'Same_district']].groupby('Same_district').count().plot.bar(ax = ax2, color = 'red', legend = False)

In [ ]:
# Barchart for gender

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))

df11_good[['gender', 'account_id']].groupby('gender').count().plot.bar(ax = ax1, color = 'grey', legend = False)
df11_bad[['gender', 'account_id']].groupby('gender').count().plot.bar(ax= ax2, color = 'red', legend = False)

In [ ]:
# histogram for owner age

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 5))

df11_good['Owner_age'].hist(ax = ax1, label = 'good loan', color = 'grey', bins = 25)
df11_bad['Owner_age'].hist(ax = ax2, label = 'bad_loan', color = 'red', bins = 25)

ax1.legend()
ax2.legend()

# Finalizing Features

In [ ]:
query = """
        SELECT account_id, amount, duration, payments, Loan_date, status, Account_date, A4, A11, A12, A13, A14, A15, A16
        FROM Loan JOIN Account USING(account_id) JOIN District USING(district_id)
        """

df = db.to_df(query)

In [ ]:
df.head()

In [ ]:
df.set_index('account_id', inplace = True)

In [ ]:
df['DID'] = (df['Loan_date'] - df['Account_date']).dt.days
df = df.rename(columns = {'A4': 'No_inhabitants', 'A11': 'Average_salary'})
df['Average_unemployment_rate'] = df[['A12', 'A13']].mean(axis = 1)
df['Average_crime_rate'] = df[['A15', 'A16']].mean(axis = 1)/df['No_inhabitants']

In [ ]:
query = """
        SELECT account_id, amount Order_amount
        FROM `Order`
        WHERE account_id IN(SELECT account_id FROM Loan);
        """

df2 = db.to_df(query)

In [ ]:
df2['Order_amount'] = df2['Order_amount'].astype('float')
df = df.join(df2.groupby('account_id').mean())
df = df.rename(columns = {'Order_amount': 'Average_order_amount'})

In [ ]:
df.head()

In [ ]:
query = """
        SELECT account_id, amount Trans_amount, balance Trans_balance
        FROM Trans
        WHERE account_id IN (SELECT account_id FROM Loan);
        """

df3 = db.to_df(query)

In [ ]:
df3.head()

In [ ]:
No_transaction = df3.groupby('account_id').count().iloc[: , 1]

In [ ]:
No_transaction.name = 'No_transaction'

In [ ]:
df3 = df3.groupby('account_id').mean()
df3.columns = ['Average_trans_amount', 'Average_trans_balance']

In [ ]:
df = df.join(df3).join(No_transaction)

In [ ]:
df.head()

In [ ]:
query = """
        SELECT account_id, Card.type Card_type
        FROM Loan JOIN Desposition USING(account_id) LEFT JOIN Card USING(disp_id)
        WHERE Desposition.type = 'OWNER';
        """

df4 = db.to_df(query)

In [ ]:
df4.head()

In [ ]:
df = df.join(df4.set_index('account_id'), how = 'left')

In [ ]:
df['Card_type'].fillna('No', inplace = True)

In [ ]:
df.head()

In [ ]:
query = """
        SELECT account_id, Loan_date, Account.district_id Account_dist_id, Client.district_id Client_dist_id, gender, birth_date
        FROM Loan JOIN Account USING(account_id) JOIN Desposition USING(account_id) JOIN Client USING(Client_id) WHERE Desposition.type = 'Owner';
        """

df5 = db.to_df(query)

In [ ]:
df5.head()

In [ ]:
df5['Same_district'] = df5['Account_dist_id'] == df5['Client_dist_id']
df5['Owner_age'] = (df5['Loan_date'] - df5['birth_date']).astype('timedelta64[M]')

In [ ]:
df = df.join(df5.set_index('account_id')[['Same_district', 'gender', 'Owner_age']])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['Default'] = df['status'].isin(['A', 'D'])

In [ ]:
df = df[['amount', 'duration', 'payments', 'DID', 'Average_order_amount',
         'Average_trans_amount', 'Average_trans_balance', 'No_transaction', 
         'Card_type', 'No_inhabitants', 'Average_salary', 'Average_unemployment_rate', 
         'Average_crime_rate', 'gender', 'Owner_age', 'Same_district', 'Default']]

In [ ]:
df.sample(10)

In [ ]:
df.shape

In [ ]:
df.columns

# Correlations

In [ ]:
df.corr()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize = (10,8))
sns.heatmap(df.corr(), cmap = 'RdYlGn', annot = True, annot_kws = {'fontsize': 7.5}, linewidths = 0.2)

# Transformation

In [ ]:
import pandas as pd

In [ ]:
for i in df.columns:
    print(i , df[i].dtype)

In [ ]:
df['payments'] = df['payments'].astype('float')

In [ ]:
df2 = df.copy()

In [ ]:
# To seperate numerical and categorical data

cat_columns = []
num_columns = []
bool_columns = []

for i in df2.columns:
    if df2[i].dtype == 'object':
        cat_columns.append(i)
    else:
        if df2[i].dtype =='bool':
            bool_columns.append(i)
        else:
            num_columns.append(i)
        
print(cat_columns,'\n\n', num_columns, '\n\n', bool_columns)

In [ ]:
df2 = pd.get_dummies(df2)

In [ ]:
df2.head()

In [ ]:
df2.columns.tolist()

In [ ]:
df2.iloc[:,-6:]

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df3 = df.copy()

In [ ]:
# To seperate numerical and categorical data

cat_columns = []
num_columns = []
bool_columns = []

for i in df3.columns:
    if df3[i].dtype == 'object':
        cat_columns.append(i)
    else:
        if df3[i].dtype =='bool':
            bool_columns.append(i)
        else:
            num_columns.append(i)
        
print(cat_columns,'\n\n', num_columns, '\n\n', bool_columns)

In [ ]:
OHE = OneHotEncoder()

OHE.fit_transform(df3[cat_columns])

In [ ]:
from sklearn.compose import ColumnTransformer

# cat_cols = []
# num_cols = []

# for i in df3.columns:
#     if df3[i].dtype == 'object' or df3[i].dtype =='bool':
#         cat_cols.append(i)
#     else:
#         num_cols.append(i)

# print(cat_cols,'\n\n', num_cols)

ColumnTransformer([('cat', OneHotEncoder(), cat_columns)]).fit_transform(df3)

In [ ]:
pd.get_dummies(df3[cat_columns]).iloc[:, :]

### Profile report

In [ ]:
# from pandas_profiling import ProfileReport

# profile = ProfileReport(df)
# profile.to_file(output_file = 'Profile_report.html')

In [ ]:
# # To veiw profile report in notebook
# profile.to_notebook_iframe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_num = pd.DataFrame(MinMaxScaler().fit_transform(df[num_columns]))

In [ ]:
df_num.head()

In [ ]:
df_cat = pd.get_dummies(df[cat_columns]).reset_index(drop = True)

In [ ]:
df_cat.head()

In [ ]:
df_bool = df[bool_columns].reset_index(drop = True)
df_bool.head()

In [ ]:
# Dataframe for machine learning

df_ml = df_num.join(df_cat).join(df_bool)

In [ ]:
df_ml

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
column_transformer = ColumnTransformer([('num_trans', MinMaxScaler(), num_columns),
                                        ('cat_trans', OneHotEncoder(), cat_columns)], 
                                       remainder = 'passthrough'
                                      )

In [ ]:
df_ml = pd.DataFrame(column_transformer.fit_transform(df3))
df_ml.head()

In [ ]:
print(len(df3.columns), df3.columns, '\n\n')
print(len(df_ml.columns), df_ml.columns)

In [ ]:
# To add feature names to the transformed dataframe

df_ml.columns = column_transformer.get_feature_names_out()
df_ml.head()

## Upscalling minority class data (default cases)

In [ ]:
# Library package for upscalling imbalanced data

In [ ]:
# pip install imbalanced-learn==0.8.0

In [ ]:
df_ml_2 = df_ml.copy()

In [ ]:
X = df_ml_2.iloc[:, :-1]
Y = df_ml_2.iloc[:, -1]

In [ ]:
# import imblearn.over_sampling

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
oversample = ADASYN(sampling_strategy = 0.8)

In [ ]:
x, y = oversample.fit_resample(X, Y)

In [ ]:
print(Y.value_counts(), '\n\n', y.value_counts())

In [ ]:
355/434

---
---

# Modelling

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y, random_state = 6)

## <span style = 'background: Pink'> KNN model </span>

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(x_train, y_train)

In [ ]:
y_pred = knn.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
class_report = classification_report(y_test, y_pred)
print(class_report)

## <span style = 'background: Pink'> Linear regression model </span>

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
class_report = classification_report(y_test, y_pred)
print(class_report)

## Cross-validation and Kfold

### Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(lr, x_train, y_train, cv = 5, scoring = 'accuracy')

In [ ]:
# gives prediction of each element in the input when it was in test split in cross-validation

from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(lr, x, y, cv = 5)
confusion_matrix(y, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y, y_pred)

### Kfold

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True)

for train_index, test_index in kfold.split(x, y):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    lr.fit(x_train, y_train)
    y_pred = lr.predict(x_test)
    print(confusion_matrix(y_test, y_pred), '\n')

## GridsearchCV

In [ ]:
from sklearn. model_selection import GridSearchCV

In [ ]:
# hyperparameters tuning

params = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8]}
          
clf = GridSearchCV(KNeighborsClassifier(), param_grid = params, 
                   cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 6), 
                   scoring = 'f1')
          
clf.fit(x_train, y_train)
          
print(clf.best_params_)

In [ ]:
clf.cv_results_

## <span style = 'background: Yellow'> Defining function for model training with GridSearchCV </span>

In [ ]:
def ml(model, folds, params = {}):
    
    global clf
    
    clf = GridSearchCV(model, param_grid = params, 
                       cv = StratifiedKFold(n_splits = folds, shuffle = True, random_state = 6),
                       scoring = 'f1')
    
    clf.fit(x_train, y_train)

In [ ]:
ml(LogisticRegression(),5)                       

clf.cv_results_

## <span style = 'background: Pink'> Random Forest model </span>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
params = {'max_depth': [10, 20, 25, 30, 35, 40],
          'criterion': ['gini', 'entropy']}

ml(RandomForestClassifier(), 3, params)

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

In [ ]:
clf2 = RandomForestClassifier(criterion = 'gini', max_depth = 25)
clf2.fit(x_train, y_train)
y_pred = clf2.predict(x_test)
accuracy_score(y_test, y_pred)

## Threshold tunning

In [ ]:
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Probability for the predicting each instance as 1

pred_prob_fr_posit = lr.predict_proba(x_test)[:,1]

### ROC_AUC curve

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
roc_auc_score(y_test, pred_prob_fr_posit)

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, pred_prob_fr_posit)

In [ ]:
sensitivity = tpr
specivity = 1- fpr

gmean = (sensitivity*specivity)**0.5

In [ ]:
threshold[gmean.argmax()]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(fpr, tpr)
plt.plot(fpr[gmean.argmax()], tpr[gmean.argmax()], marker = 'o')

In [ ]:
# Youden's J statistics

Y_stat = tpr-fpr
threshold[Y_stat.argmax()]

In [ ]:
plt.plot(fpr, tpr)
plt.plot(fpr[Y_stat.argmax()], tpr[Y_stat.argmax()], marker = 'o')

### Precision- Recall curve

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, threshold = precision_recall_curve(y_test, pred_prob_fr_posit)

fscore = 2*precision*recall/(precision+recall)

In [ ]:
fscore.argmax()

In [ ]:
plt.plot(precision, recall)
plt.plot(precision[fscore.argmax()], recall[fscore.argmax()], marker = 'o')

### Checking scores at different thresolds for the model

In [ ]:
import numpy as np

threshold = np.arange(0, 1.1, 0.1)
threshold

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
lr.fit(x_train, y_train)

# To check score on different thresholds
# lr.predict_proba(x_test)[:, 1] >= i).astype('int')

for i in threshold:
    print(confusion_matrix(y_test, (lr.predict_proba(x_test)[:, 1] >= i).astype('int')))

## <span style = 'background: Pink'> XGboost model </span>

In [ ]:
# pip install xgboost

In [ ]:
from xgboost import XGBClassifier

---
---

### <span style = 'background: Yellow'> Running different models with default parameters on the training set </span>

In [ ]:
for i in [KNeighborsClassifier(), LogisticRegression(), RandomForestClassifier(), XGBClassifier()]:
    i.fit(x_train, y_train)
    y_pred = i.predict(x_test)
    print(f1_score(y_test, y_pred), i)

---
---

In [ ]:
xgb = XGBClassifier()

In [ ]:
xgb.fit(x_train, y_train)

In [ ]:
y_pred = xgb.predict(x_test)

confusion_matrix(y_test, y_pred)

### GridSearchCV on XGboost

In [ ]:
%time
params = {'max_depth': [4, 5, 6, 7, 8],
         'learning_rate': [0.05, 0.01, 0.1],
         'gamma': [0, 0.25, 1],
         'reg_lambda': [0, 1, 10],
         'subsample': [0.5, 0.75, 1],
         'colsample_bytree': [0.5, 0.75, 1]}

ml(XGBClassifier(), 4, params)

In [ ]:
clf.cv_results_

In [ ]:
clf.best_params_

### <span style = 'background: Red'> Training final/best model from GridSearchCV </span>

In [ ]:
xgb = XGBClassifier(max_depth = 7, colsample_bytree = 1, gamma = 0.25, learning_rate = 0.1, reg_lambda = 1, subsample = 1)

In [ ]:
xgb.fit(x_train, y_train)
y_pred = xgb.predict(x_test)
confusion_matrix(y_test, y_pred)

In [ ]:
# xgb = XGBClassifier(max_depth = 7)
# xgb.fit(x_train, y_train)

# To check score on different thresholds
threshold = np.arange(0.1, 0.9, 0.1)

for i in threshold:
    print(f1_score(y_test, (xgb.predict_proba(x_test)[:, 1] >= i).astype('int')), i)

# Saving model

In [ ]:
import pickle

In [ ]:
pickle_out = open("xgb.pkl", mode = 'wb')

pickle.dump(xgb, pickle_out)

pickle_out.close()